In [1]:
import pandas as pd

# 파일 경로
train_path = "train.csv"
test_path = "test.csv"

# 데이터 로드
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# 데이터 구조 확인
train_info = train_df.info()
test_info = test_df.info()

# train 데이터 상위 5개 확인
train_head = train_df.head()

train_info, test_info, train_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   UID                10000 non-null  object 
 1   주거 형태              10000 non-null  object 
 2   연간 소득              10000 non-null  float64
 3   현재 직장 근속 연수        10000 non-null  object 
 4   체납 세금 압류 횟수        10000 non-null  float64
 5   개설된 신용계좌 수         10000 non-null  int64  
 6   신용 거래 연수           10000 non-null  float64
 7   최대 신용한도            10000 non-null  float64
 8   신용 문제 발생 횟수        10000 non-null  int64  
 9   마지막 연체 이후 경과 개월 수  10000 non-null  int64  
 10  개인 파산 횟수           10000 non-null  int64  
 11  대출 목적              10000 non-null  object 
 12  대출 상환 기간           10000 non-null  object 
 13  현재 대출 잔액           10000 non-null  float64
 14  현재 미상환 신용액         10000 non-null  float64
 15  월 상환 부채액           10000 non-null  float64
 16  신용 점수              1000

(None,
 None,
            UID 주거 형태      연간 소득 현재 직장 근속 연수  체납 세금 압류 횟수  개설된 신용계좌 수  \
 0  TRAIN_00000    자가  1941337.5      10년 이상          0.0           9   
 1  TRAIN_00001    월세  1979505.0      10년 이상          0.0           5   
 2  TRAIN_00002    월세  1356381.0          4년          0.0          12   
 3  TRAIN_00003    월세  1049017.5          6년          0.0          15   
 4  TRAIN_00004    월세  4320217.5          2년          0.0          11   
 
    신용 거래 연수   최대 신용한도  신용 문제 발생 횟수  마지막 연체 이후 경과 개월 수  개인 파산 횟수  대출 목적  \
 0      13.4  400597.5            0                 24         1  부채 통합   
 1      15.1  360679.5            0                 11         0  부채 통합   
 2      18.8  491770.5            1                 74         3  부채 통합   
 3      14.8  411546.0            1                 22         1  부채 통합   
 4      26.1  895288.5            0                 32         0  부채 통합   
 
   대출 상환 기간   현재 대출 잔액  현재 미상환 신용액  월 상환 부채액  신용 점수  채무 불이행 여부  
 0    단기 상환   390903.0    225

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# 1. 불필요한 컬럼 제거
train_df = train_df.drop(columns=["UID"])
test_UID=test_df.pop('UID')

# 2. 범주형 변수 처리 (레이블 인코딩)
categorical_cols = ["주거 형태", "현재 직장 근속 연수", "대출 목적", "대출 상환 기간"]
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    label_encoders[col] = le

# 3. 특징(X)과 타겟(y) 분리
X = train_df.drop(columns=["채무 불이행 여부"])
y = train_df["채무 불이행 여부"]
X_test = test_df.copy()

# 4. 데이터 분할 (학습용/검증용)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 5. 스케일링 (표준화)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

# 데이터 전처리 완료
X_train_scaled.shape, X_valid_scaled.shape, X_test_scaled.shape

((8000, 16), (2000, 16), (2062, 16))

In [3]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# XGBoost 모델 학습
model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric="logloss")
model.fit(X_train_scaled, y_train)

# 검증 데이터에 대한 예측
y_valid_pred = model.predict_proba(X_valid_scaled)[:, 1]

# ROC-AUC 평가
roc_auc = roc_auc_score(y_valid, y_valid_pred)
roc_auc

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:48:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


0.7058504176326879

In [4]:
from sklearn.model_selection import RandomizedSearchCV

# XGBoost 하이퍼파라미터 튜닝을 위한 파라미터 그리드
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.7, 0.8, 0.9],
    "colsample_bytree": [0.7, 0.8, 0.9]
}

# RandomizedSearchCV 실행
xgb = XGBClassifier(random_state=42, eval_metric="logloss", use_label_encoder=False)
random_search = RandomizedSearchCV(xgb, param_grid, n_iter=10, scoring="roc_auc", cv=3, random_state=42, n_jobs=-1)
random_search.fit(X_train_scaled, y_train)

# 최적 파라미터 및 성능 확인
best_params = random_search.best_params_
best_score = random_search.best_score_

best_params, best_score

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [07:48:45] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


({'subsample': 0.9,
  'n_estimators': 200,
  'max_depth': 3,
  'learning_rate': 0.05,
  'colsample_bytree': 0.8},
 0.7427826617478358)

In [5]:
# 최적 하이퍼파라미터로 XGBoost 모델 재학습
best_model = XGBClassifier(**best_params, random_state=42, eval_metric="logloss")
best_model.fit(X_train_scaled, y_train)

# 검증 데이터 평가
y_valid_pred = best_model.predict_proba(X_valid_scaled)[:, 1]
roc_auc_final = roc_auc_score(y_valid, y_valid_pred)

# 테스트 데이터 예측
y_test_pred = best_model.predict_proba(X_test_scaled)[:, 1]

roc_auc_final

0.750733137829912

In [8]:
import pandas as pd

# 예측 결과 저장
result10 = pd.DataFrame({"UID": test_UID, "채무 불이행 확률": y_test_pred})
result10.to_csv("result10.csv", index=False)